In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *
import collections

pd.set_option('display.max_rows', None)

tokenizer = Tokenizer()
char_filters = [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(u'蛇の目', u'janome')]
token_filters = [CompoundNounFilter(), POSStopFilter(['記号','助詞', '助動詞','動詞','連体詞','接続詞']), LowerCaseFilter()]
analyzer = Analyzer(char_filters=char_filters, tokenizer=tokenizer, token_filters=token_filters)

# scraping
r = requests.get("URL")
soup = BeautifulSoup(r.content, "html.parser")
articles = soup.find_all('article')
all_paragraph = ''
for article in articles:
    paragraphs =  article.find_all('p')
    for p in paragraphs:
        all_paragraph += p.text

c = collections.Counter(token.base_form for token in analyzer.analyze(all_paragraph))

df = pd.DataFrame(c.most_common(), columns=['word', 'count'])
df